# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw

In [2]:
data_folder='/Users/izuard/Desktop/AI/Sources/data_lab4/'

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [3]:
reviews_pd=pd.read_csv(data_folder+'reviews_sample.csv',index_col=0)
recipes_pd=pd.read_csv(data_folder+'recipes_sample.csv').drop(['contributor_id','n_steps'],axis=1)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [4]:
part_reviews_pd=reviews_pd.sample(frac=0.05)
part_recipes_pd=recipes_pd.sample(frac=0.05)

with pd.ExcelWriter(data_folder+'recipes.xlsx') as writer:
    part_recipes_pd.to_excel(writer,sheet_name='recipes')
    part_reviews_pd.to_excel(writer,sheet_name='reviews')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [5]:
xl_book=xw.Book(data_folder+'recipes.xlsx')
recipes_sheet=xl_book.sheets['recipes']
reviews_sheet=xl_book.sheets['reviews']

In [6]:
recipes_sheet['G1:G1501'].value=part_recipes_pd['minutes']*60
recipes_sheet['G1'].value='seconds_assign'

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [7]:
recipes_sheet['H2:H1501'].formula='=D2*60'
recipes_sheet['H1'].value='seconds_formula'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [41]:
from xlwings import constants

In [8]:
recipes_sheet['G1:H1'].api.font_object.font_style.set('bold')
# recipes_sheet['G1:H1'].api.HorizontalAlignment=constants.HAlign.xlHAlignLeft

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [43]:
for cell in recipes_sheet['D2:D1501']:
    if cell.value<5:
        cell.color=(124,252,0)
    elif cell.value<=10:
        cell.color=(204,204,0)
    else:
        cell.color=(233,150,122)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [45]:
recipes_sheet['I2:I1501'].formula='=countif(reviews!C:C,C2)'
recipes_sheet['I1'].value='n_reviews'
xl_book.save()